In [1]:
import mujoco
import numpy as np
import stac_mjx
from pathlib import Path
import h5py
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import re
import time
import imageio
import mediapy as media
import os

/opt/anaconda3/envs/stac-mjx-env/lib/python3.11/site-packages/jaxopt/__init__.py:59: DeprecationWarning: JAXopt is no longer maintained. See https://docs.jax.dev/en/latest/ for alternatives.
  warnings.warn(


In [2]:
# glfw is faster, but use osmesa if glfw not available
%env MUJOCO_GL=glfw
%env PYOPENGL_PLATFORM=glfw

# Disable jax's memory preallocation if you're running multiple notebooks using jax
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = "false" 

base_path = Path.cwd()

print(base_path)

env: MUJOCO_GL=glfw
env: PYOPENGL_PLATFORM=glfw
/Users/rusham/Documents/TalmoLab/GitRepos/stac-mjx


In [3]:

# Load configs
cfg = stac_mjx.load_configs(base_path / "configs")

# Load data
kp_data, sorted_kp_names = stac_mjx.load_mocap(cfg, base_path)


Config loaded and validated.


In [4]:
kp_data.shape

(16875, 150)

In [5]:
# Run stac
fit_path, ik_only_path = stac_mjx.run_stac(
 cfg,
 kp_data, 
 sorted_kp_names, 
 base_path=base_path
)

/Users/rusham/Documents/TalmoLab/GitRepos/stac-mjx/stac_mjx/utils.py:16: DeprecationWarning: jax.lib.xla_bridge.get_backend is deprecated; use jax.extend.backend.get_backend.
  if xla_bridge.get_backend().platform == "gpu":


Running fit. Mocap data shape: (111, 150)
ROOT_OPTIMIZATION_KEYPOINT not specified, skipping Root Optimization.
Calibration iteration: 1/30
Pose Optimization:
Pose Optimization done in 16.89319682121277
Mean: 0.04474109411239624
Standard deviation: 0.09568517655134201
starting offset optimization
Begining offset optimization:
Final error of 0.00034491001861169934
offset optimization finished in 4.2228498458862305
Calibration iteration: 2/30
Pose Optimization:
Pose Optimization done in 0.5849800109863281
Mean: 0.01951492391526699
Standard deviation: 0.02905970811843872
starting offset optimization
Begining offset optimization:
Final error of 0.0002008728333748877
offset optimization finished in 2.9805948734283447
Calibration iteration: 3/30
Pose Optimization:
Pose Optimization done in 0.6204869747161865
Mean: 0.0200102087110281
Standard deviation: 0.03161882609128952
starting offset optimization
Begining offset optimization:
Final error of 5.613121174974367e-05
offset optimization finis

In [6]:
from stac_mjx.viz import viz_stac

cfg = stac_mjx.load_configs(base_path / "configs")

data_path = base_path / "stick_fit_offsets.h5"
n_frames = 110
save_path = base_path / "videos/render.mp4"

# Call mujoco_viz
cfg, frames = viz_stac(data_path=data_path, 
                  n_frames=n_frames, 
                  save_path=save_path, 
                  start_frame=0,  
                  base_path=Path.cwd(),
                  width=640,
                  height=480,
                  )

Config loaded and validated.


0it [00:00, ?it/s]/opt/anaconda3/envs/stac-mjx-env/lib/python3.11/subprocess.py:1885: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
110it [00:01, 72.14it/s]


In [7]:
# Show the video in the notebook (it is also saved to the save_path)
media.show_video(frames, fps=cfg.model.RENDER_FPS)

This browser does not support the video tag.
